In [114]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [115]:
widsDf = pd.read_csv("/kaggle/input/widsdatathon2022/train.csv")
widsDf.head()

In [116]:
widsDf.shape

In [117]:
widsDf.info()

In [118]:
widsDf.describe()

In [119]:
widsDf.isna().sum()

In [120]:
[(column, widsDf[column].isna().sum()) for column in widsDf.columns]

In [121]:
widsDf.drop(["direction_max_wind_speed","direction_peak_wind_speed","max_wind_speed","days_with_fog"], inplace=True, axis=1)

In [122]:
widsDf.columns

In [123]:
widsDf.hist()

In [124]:
widsDf.info()

In [125]:
widsDf['State_Factor'].value_counts()

In [126]:
widsDf['building_class'].value_counts()

In [127]:
widsDf['facility_type'].value_counts()

In [128]:
# Import label encoder 
from sklearn import preprocessing
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'Country'. 
widsDf['State_Factor']= label_encoder.fit_transform(widsDf['State_Factor']) 
widsDf['building_class']= label_encoder.fit_transform(widsDf['building_class']) 
widsDf['facility_type']= label_encoder.fit_transform(widsDf['facility_type'])
widsDf.head()

In [129]:
widsDf_corr=widsDf.corr()
widsDf_corr.style.background_gradient(cmap="cool")

In [130]:
import matplotlib.pyplot as plt
import seaborn as sns
# constructing heap to see correlation
#plt.figure(figsize=(8,8))
sns.heatmap(widsDf_corr, cbar=True, square=True, fmt='.1f', annot=True,annot_kws={'size':8},cmap='Blues')

In [131]:
features = widsDf.drop('site_eui',axis=1)
outputLabel = widsDf['site_eui']
print(outputLabel)

In [132]:
from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
# Imputation transformer for completing missing values.
imp_mean.fit(features)
features = imp_mean.transform(features)
features = pd.DataFrame(features)
features.head

In [133]:
#sns.distplot(widsDf['energy_star_rating'])
sns.distplot(widsDf['year_built'])

In [134]:
# define min max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# transform data
scaled = scaler.fit_transform(features)
print(scaled)

In [135]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled, outputLabel, test_size=0.25, random_state=46)

In [136]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10,5))
plt.title('energy_star_rating & site_eui')
plt.xlabel('energy_star_rating')
plt.ylabel('site_eui')
sns.scatterplot(widsDf['energy_star_rating'],widsDf['site_eui'])
plt.show()

In [137]:
#Prepare a Linear Regression Model
from sklearn.linear_model import LinearRegression

reg=LinearRegression()
reg.fit(X_train,y_train)
y_pred=reg.predict(X_test)


In [138]:
import sklearn
import math
mse = sklearn.metrics.mean_squared_error(y_test,y_pred)
rmse = math.sqrt(mse)
rmse

In [139]:
import xgboost

xgboost_model = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.03, gamma=0, subsample=0.75,
                           colsample_bytree=0.4, max_depth=3)
xgboost_model.fit(X_train,y_train)

In [140]:
predictions = xgboost_model.predict(X_test)
mse_xg = sklearn.metrics.mean_squared_error(y_test,predictions)
rmse_xg = math.sqrt(mse_xg)
rmse_xg

In [141]:
# from sklearn.model_selection import GridSearchCV
# xgb = xgboost.XGBRegressor()
# parameters = {'objective':['reg:linear'],
#               'learning_rate': [.02, .05, 0.01], #so called `eta` value
#               'max_depth': [3,5],
#               'min_child_weight': [4],
#               'subsample': [0.7],
#               'colsample_bytree': [0.4],
#               'n_estimators': [1000],
#               'reg_alpha': [0.4],
#               'reg_lambda': [2e-08]
# }
# xgb_grid = GridSearchCV(xgb,
#                         parameters,
#                         cv = 5,
#                         verbose=True)

# xgb_grid.fit(X_train,y_train)

# print(xgb_grid.best_score_)
# print(xgb_grid.best_params_)

In [142]:
from sklearn.model_selection import KFold

In [143]:
kfold = KFold(n_splits=6)


In [144]:
from lightgbm import LGBMRegressor

In [145]:
y_train.shape

In [146]:
    
xgboost_mod = xgboost.XGBRegressor(colsample_bytree=0.4, learning_rate= 0.05, max_depth= 5, min_child_weight= 4,
                          n_estimators= 1000, reg_alpha=0.4, reg_lambda=2e-08, subsample= 0.7)
xgboost_mod.fit(X_train,y_train)

In [ ]:
y_train

In [147]:
predictions_mod = xgboost_mod.predict(X_test)
mse_xg = sklearn.metrics.mean_squared_error(y_test,predictions_mod)
rmse_xg = math.sqrt(mse_xg)
rmse_xg